### Load preprocessed data

In [1]:
import numpy as np
fh = np.load('../data/dataset.npz')
# We have a bunch of feature columns and last column is the y-target
train_x = fh['train_x'].astype(np.int64)
train_y = fh['train_y']

test_x = fh['test_x'].astype(np.int64)
test_y = fh['test_y']


n_user = int(fh['n_user'])
n_item = int(fh['n_item'])

### Define the MF Model

In [7]:
import torch
from torch import nn
import torch.nn.functional as F

def l2_regularize(array):
    loss = torch.sum(array ** 2.0)
    return loss


class MF(nn.Module):
    itr = 0
    
    def __init__(self, n_user, n_item, k=18, c_vector=1.0, c_bias=1.0, writer=None):
        super(MF, self).__init__()
        self.writer = writer
        self.k = k
        self.n_user = n_user
        self.n_item = n_item
        self.c_bias = c_bias
        self.c_vector = c_vector
        self.user = nn.Embedding(n_user, k)
        self.item = nn.Embedding(n_item, k)
        
        # We've added new terms here:
        self.bias_user = nn.Embedding(n_user, 1)
        self.bias_item = nn.Embedding(n_item, 1)
        self.bias = nn.Parameter(torch.ones(1))

    
    def __call__(self, train_x):
        user_id = train_x[:, 0]
        item_id = train_x[:, 1]
        vector_user = self.user(user_id)
        vector_item = self.item(item_id)
        bias_user = self.bias_user(user_id).squeeze()
        bias_item = self.bias_item(item_id).squeeze()
        biases = (self.bias + bias_user + bias_item)
        ui_interaction = torch.sum(vector_user * vector_item, dim=1)
        prediction = ui_interaction + biases
        return prediction
    
    def loss(self, prediction, target):
        loss_mse = F.mse_loss(prediction, target)
        prior_bias_user =  l2_regularize(self.bias_user.weight) * self.c_bias
        prior_bias_user = l2_regularize(self.bias_item.weight) * self.c_bias
        prior_user =  l2_regularize(self.user.weight) * self.c_vector
        prior_item = l2_regularize(self.item.weight) * self.c_vector
        total = loss_mse + prior_user + prior_item
        for name, var in locals().items():
            if type(var) is torch.Tensor and var.nelement() == 1 and self.writer is not None:
                self.writer.add_scalar(name, var, self.itr)
        return total

### Train model

In [8]:
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Loss
from tensorboardX import SummaryWriter
from ignite.metrics import MeanSquaredError

from loader import Loader

#### Hyperparameters

In [9]:
n_epochs = 10
k = 10
c_bias = 1e-6
c_vector = 1e-6

In [10]:
writer = SummaryWriter()
model = MF(n_user, n_item, writer=writer, k=k, c_bias=c_bias, c_vector=c_vector)
optimizer = torch.optim.Adam(model.parameters())
trainer = create_supervised_trainer(model, optimizer, model.loss)
metrics = {'accuracy': MeanSquaredError()}
evaluat = create_supervised_evaluator(model, metrics=metrics)
train_loader = Loader(train_x, train_y, batchsize=1024)
test_loader = Loader(test_x, test_y, batchsize=1024)


def log_training_loss(engine, log_interval=400):
    epoch = engine.state.epoch
    itr = engine.state.iteration
    fmt = "Epoch[{}] Iteration[{}/{}] Loss: {:.2f}"
    msg = fmt.format(epoch, itr, len(train_loader), engine.state.output)
    model.itr = itr
    if itr % log_interval == 0:
        print(msg)

trainer.add_event_handler(event_name=Events.ITERATION_COMPLETED, handler=log_training_loss)

def log_validation_results(engine):
    evaluat.run(test_loader)
    metrics = evaluat.state.metrics
    avg_accuracy = metrics['accuracy']
    print("Epoch[{}] Validation MSE: {:.2f} "
          .format(engine.state.epoch, avg_accuracy))
    writer.add_scalar("validation/avg_accuracy", avg_accuracy, engine.state.epoch)

trainer.add_event_handler(event_name=Events.EPOCH_COMPLETED, handler=log_validation_results)


model

MF(
  (user): Embedding(6041, 10)
  (item): Embedding(3953, 10)
  (bias_user): Embedding(6041, 1)
  (bias_item): Embedding(3953, 1)
)

#### Run model

In [11]:
trainer.run(train_loader, max_epochs=50)

Epoch[1] Iteration[400/879] Loss: 14.47
Epoch[1] Iteration[800/879] Loss: 11.86
Epoch[1] Validation MSE: 10.83 
Epoch[2] Iteration[1200/879] Loss: 8.89
Epoch[2] Iteration[1600/879] Loss: 6.64
Epoch[2] Validation MSE: 6.72 
Epoch[3] Iteration[2000/879] Loss: 5.38
Epoch[3] Iteration[2400/879] Loss: 4.55
Epoch[3] Validation MSE: 4.63 
Epoch[4] Iteration[2800/879] Loss: 4.20
Epoch[4] Iteration[3200/879] Loss: 3.46
Epoch[4] Validation MSE: 3.37 
Epoch[5] Iteration[3600/879] Loss: 3.04
Epoch[5] Iteration[4000/879] Loss: 2.83
Epoch[5] Validation MSE: 2.54 
Epoch[6] Iteration[4400/879] Loss: 2.26
Epoch[6] Iteration[4800/879] Loss: 2.25
Epoch[6] Iteration[5200/879] Loss: 1.93
Epoch[6] Validation MSE: 2.00 
Epoch[7] Iteration[5600/879] Loss: 1.66
Epoch[7] Iteration[6000/879] Loss: 1.63
Epoch[7] Validation MSE: 1.64 
Epoch[8] Iteration[6400/879] Loss: 1.40
Epoch[8] Iteration[6800/879] Loss: 1.36
Epoch[8] Validation MSE: 1.40 
Epoch[9] Iteration[7200/879] Loss: 1.22
Epoch[9] Iteration[7600/879] Lo

KeyboardInterrupt: 